In [1]:
import os 
os.chdir("/home/minasian/PycharmProjects/AutoML")
os.getcwd()
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (11, 5)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import argparse
import os
import numpy as np
import pandas as pd
import pickle
import time
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.preprocessing import StandardScaler
from examples.baseline.utils import transform_datetime_features
import matplotlib.pyplot as plt

In [3]:
TIME_LIMIT = int(os.environ.get('TIME_LIMIT', 5*60))


In [4]:
start_time = time.time()
test_csv="sdsj2018_automl_check_datasets/check_1_r/test.csv"
model_dir="examples/baseline/models"
mode='regression' if test_csv.split("/")[-2].split("_")[-1]=='r' else 'classification'
df = pd.read_csv(test_csv)
df.shape

(172, 41)

In [5]:
model_config_filename = os.path.join(model_dir, 'model_config.pkl')
with open(model_config_filename, 'rb') as fin:
        model_config = pickle.load(fin)
model_config_filename

'examples/baseline/models/model_config.pkl'

In [6]:
df = pd.read_csv(test_csv)

In [7]:
df

,datetime_0,number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,...,number_30,number_31,number_32,number_33,number_34,number_35,number_36,number_37,number_38,line_id
0,2018-05-21,0.565964,0.882359,0.145729,0.699714,0.817038,0.000000,0.069541,0.031546,0.034921,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-12,0.966155,0.473820,0.133166,0.707340,0.603106,0.181818,0.383866,0.132492,0.165079,...,0,0,0,0,0,0,0,0,0,2
2,2018-01-29,0.249934,0.689851,0.221106,0.716873,0.604604,0.181818,0.076495,0.069401,0.069841,...,0,0,1,0,0,0,0,0,0,6
3,2018-01-24,0.683120,0.693576,0.115578,0.701621,0.603106,0.181818,0.052851,0.069401,0.076190,...,0,0,0,0,0,0,0,0,0,10
4,2018-01-27,0.138409,0.688569,0.195980,0.713060,0.603608,0.181818,0.332406,0.066246,0.069841,...,0,0,0,0,0,0,0,0,0,13
5,2018-04-29,0.467403,0.675272,0.133166,0.692088,0.815747,0.000000,0.581363,0.041009,0.034921,...,0,0,0,0,0,0,0,0,0,14
6,2018-01-20,0.575102,0.686199,0.135678,0.707340,0.603106,0.181818,0.282337,0.072555,0.082540,...,0,0,0,0,0,0,0,0,0,15
7,2018-02-13,0.589767,0.796630,0.090452,0.684461,0.607090,0.090909,0.058414,0.018927,0.012698,...,0,0,0,0,0,0,0,0,0,21
8,2018-01-21,0.575102,0.686199,0.135678,0.707340,0.603106,0.181818,0.282337,0.072555,0.082540,...,0,0,0,0,0,0,0,0,0,33
9,2018-04-06,0.405133,0.764478,0.148241,0.689228,0.814751,0.000000,0.265647,0.031546,0.028571,...,0,0,0,0,0,0,0,0,0,38
